In [6]:
import json
import os
import glob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
from datetime import datetime
from tqdm import tqdm  # progress bar

In [7]:
def setup_finbert():
    """Initialize FinBERT model and tokenizer"""
    print("🔹 Loading FinBERT model...")
    model_name = "ProsusAI/finbert"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.eval()
    return tokenizer, model


def truncate_text_bytes(text, max_bytes=2000):
    """Ensure text fits under max_bytes (UTF-8 safe)"""
    encoded = text.encode("utf-8")
    if len(encoded) <= max_bytes:
        return text
    truncated = encoded[:max_bytes]
    # Ensure we don't cut off in the middle of a UTF-8 character
    return truncated.decode("utf-8", errors="ignore")


def chunk_text(text, max_length=512):
    """Split text into chunks that fit FinBERT's token limit"""
    sentences = re.split(r'[.!?]+', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length * 4:
            current_chunk += sentence + ". "
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks


def analyze_with_finbert(text, tokenizer, model):
    """Analyze text with FinBERT and return sentiment summary"""
    text = truncate_text_bytes(text, 2000)

    if not text or len(text.strip()) < 10:
        return {
            "sentiment": "neutral",
            "confidence": 0.0,
            "summary": "Insufficient text for analysis"
        }

    chunks = chunk_text(text)
    sentiments, confidences = [], []

    for chunk in chunks[:10]:  # Limit to 10 chunks max
        try:
            inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=512, padding=True)
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            sentiment_idx = torch.argmax(predictions, dim=-1).item()
            confidence = torch.max(predictions).item()
            sentiment_labels = ["negative", "neutral", "positive"]
            sentiments.append(sentiment_labels[sentiment_idx])
            confidences.append(confidence)
        except Exception:
            continue

    if not sentiments:
        return {
            "sentiment": "neutral",
            "confidence": 0.0,
            "summary": "Analysis failed"
        }

    sentiment_counts = {"negative": 0, "neutral": 0, "positive": 0}
    for s in sentiments:
        sentiment_counts[s] += 1

    overall_sentiment = max(sentiment_counts, key=sentiment_counts.get)
    avg_confidence = sum(confidences) / len(confidences)

    summary = (
        f"FinBERT analysis on {len(chunks)} text segments shows {overall_sentiment.upper()} sentiment "
        f"({avg_confidence:.2f} confidence). "
        f"Distribution: {sentiment_counts['negative']} negative, {sentiment_counts['neutral']} neutral, "
        f"{sentiment_counts['positive']} positive."
    )

    return {
        "sentiment": overall_sentiment,
        "confidence": avg_confidence,
        "summary": summary,
        "segment_count": len(chunks),
        "sentiment_distribution": sentiment_counts
    }


def process_file(file_path, tokenizer, model):
    """Process a single JSON file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        part2item7_text = data.get('data', {}).get('sections', {}).get('part2item7', {}).get('text', '')
        if not part2item7_text:
            return False, "no_part2item7"

        analysis = analyze_with_finbert(part2item7_text, tokenizer, model)

        data['finberg'] = {
            "summary": analysis['summary'],
            "sentiment": analysis['sentiment'],
            "confidence": analysis['confidence'],
            "segment_count": analysis.get('segment_count', 0),
            "sentiment_distribution": analysis.get('sentiment_distribution', {}),
            "generated_by": "finbert",
            "source_section": "part2item7",
            "ticker": data.get('ticker', 'UNKNOWN'),
            "analysis_date": datetime.now().isoformat()
        }

        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

        return True, analysis['sentiment']

    except Exception as e:
        return False, str(e)


def main():
    """Main function to process all JSON files in completed_data_plus directory"""
    data_dir = "/home/sagemaker-user/shared/Project/advanced_tags/completed_data_plus"
    json_files = glob.glob(os.path.join(data_dir, "*.json"))
    print(f"📂 Found {len(json_files)} JSON files to process in {data_dir}")

    if not json_files:
        print(f"No JSON files found in {data_dir}.")
        return

    tokenizer, model = setup_finbert()
    successful, failed = 0, 0

    for file_path in tqdm(json_files, desc="Processing files", unit="file"):
        ok, result = process_file(file_path, tokenizer, model)
        if ok:
            successful += 1
        else:
            failed += 1

    print(f"\n✅ Processing complete: {successful} succeeded, ❌ {failed} failed.")


# if __name__ == "__main__":
#     main()

In [8]:
# Tester le chemin et exécuter l'analyse FinBERT
import os
data_dir = "/home/sagemaker-user/shared/Project/advanced_tags/completed_data_plus"
print(f"Directory exists: {os.path.exists(data_dir)}")
if os.path.exists(data_dir):
    files = os.listdir(data_dir)
    json_files = [f for f in files if f.endswith('.json')]
    print(f"Found {len(json_files)} JSON files")
    main()
else:
    print("Directory not found")

Directory exists: True
Found 490 JSON files
📂 Found 490 JSON files to process in /home/sagemaker-user/shared/Project/advanced_tags/completed_data_plus
🔹 Loading FinBERT model...


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Processing files:   0%|          | 0/490 [00:00<?, ?file/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Processing files: 100%|██████████| 490/490 [06:52<00:00,  1.19file/s]


✅ Processing complete: 428 succeeded, ❌ 62 failed.
